#Importación de librerías

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
!pip install kaggle
!pip install optuna xgboost

In [60]:
import optuna

# Carga de los datos

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.85GB/s]


In [4]:
!unzip -l udea-ai-4-eng-20251-pruebas-saber-pro-colombia

Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
  4716673  2025-03-25 16:54   submission_example.csv
 59185250  2025-03-25 16:54   test.csv
143732449  2025-03-25 16:54   train.csv
---------                     -------
207634372                     3 files


In [5]:
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia

Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [6]:
!ls -lash *.csv

4.5M -rw-r--r-- 1 root root 4.5M Mar 25 16:54 submission_example.csv
 57M -rw-r--r-- 1 root root  57M Mar 25 16:54 test.csv
138M -rw-r--r-- 1 root root 138M Mar 25 16:54 train.csv


# Exploración

In [37]:
dtr = pd.read_csv('train.csv')

In [38]:
dtr.shape

(692500, 21)

In [39]:
dtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              692500 non-null  int64  
 1   PERIODO                         692500 non-null  int64  
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object 
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object 
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object 
 7   FAMI_TIENEINTERNET              665871 non-null  object 
 8   FAMI_EDUCACIONPADRE             669322 non-null  object 
 9   FAMI_TIENELAVADORA              652727 non-null  object 
 10  FAMI_TIENEAUTOMOVIL             648877 non-null  object 
 11  ESTU_PRIVADO_LIBERTAD           692500 non-null  object 
 12  ESTU_PAGOMATRICU

In [40]:
dtr.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [41]:
for i in dtr.columns:
    print ("%20s"%i, np.sum(dtr[i].isna()))

                  ID 0
             PERIODO 0
 ESTU_PRGM_ACADEMICO 0
ESTU_PRGM_DEPARTAMENTO 0
ESTU_VALORMATRICULAUNIVERSIDAD 6287
ESTU_HORASSEMANATRABAJA 30857
FAMI_ESTRATOVIVIENDA 32137
  FAMI_TIENEINTERNET 26629
 FAMI_EDUCACIONPADRE 23178
  FAMI_TIENELAVADORA 39773
 FAMI_TIENEAUTOMOVIL 43623
ESTU_PRIVADO_LIBERTAD 0
ESTU_PAGOMATRICULAPROPIO 6498
FAMI_TIENECOMPUTADOR 38103
FAMI_TIENEINTERNET.1 26629
 FAMI_EDUCACIONMADRE 23664
  RENDIMIENTO_GLOBAL 0
              coef_1 0
              coef_2 0
              coef_3 0
              coef_4 0


In [42]:
dtr.ESTU_HORASSEMANATRABAJA.value_counts()

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,249352
0,116550
Entre 11 y 20 horas,115857
Entre 21 y 30 horas,92693
Menos de 10 horas,87191


In [43]:
dtr.RENDIMIENTO_GLOBAL.value_counts()

,count
RENDIMIENTO_GLOBAL,
alto,175619
bajo,172987
medio-bajo,172275
medio-alto,171619


In [44]:
dtr.ESTU_PRIVADO_LIBERTAD.value_counts()

,count
ESTU_PRIVADO_LIBERTAD,
N,692466
S,34


# Preprocesado

In [45]:
dtr.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


**Analizamos valores faltantes y observamos los datos de algunas variables**

In [16]:
for i in dtr.columns:
    print ("%20s"%i, np.sum(dtr[i].isna()))

                  ID 0
             PERIODO 0
 ESTU_PRGM_ACADEMICO 0
ESTU_PRGM_DEPARTAMENTO 0
ESTU_VALORMATRICULAUNIVERSIDAD 6287
ESTU_HORASSEMANATRABAJA 30857
FAMI_ESTRATOVIVIENDA 32137
  FAMI_TIENEINTERNET 26629
 FAMI_EDUCACIONPADRE 23178
  FAMI_TIENELAVADORA 39773
 FAMI_TIENEAUTOMOVIL 43623
ESTU_PRIVADO_LIBERTAD 0
ESTU_PAGOMATRICULAPROPIO 6498
FAMI_TIENECOMPUTADOR 38103
FAMI_TIENEINTERNET.1 26629
 FAMI_EDUCACIONMADRE 23664
  RENDIMIENTO_GLOBAL 0
              coef_1 0
              coef_2 0
              coef_3 0
              coef_4 0


In [17]:
dtr.FAMI_EDUCACIONMADRE.value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,141744
Primaria incompleta,99420
Técnica o tecnológica completa,89542
Educación profesional completa,85326
Secundaria (Bachillerato) incompleta,81012
Primaria completa,56125
Postgrado,46246
Técnica o tecnológica incompleta,27533
Educación profesional incompleta,22470


In [18]:
dtr.FAMI_ESTRATOVIVIENDA.value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,232671
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


In [19]:
dtr.ESTU_HORASSEMANATRABAJA.value_counts()

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,249352
0,116550
Entre 11 y 20 horas,115857
Entre 21 y 30 horas,92693
Menos de 10 horas,87191


In [20]:
dtr.ESTU_VALORMATRICULAUNIVERSIDAD.value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
Entre 1 millón y menos de 2.5 millones,204048
Entre 2.5 millones y menos de 4 millones,127430
Menos de 500 mil,80263
Entre 500 mil y menos de 1 millón,78704
Entre 4 millones y menos de 5.5 millones,69736
Más de 7 millones,68014
Entre 5.5 millones y menos de 7 millones,38490
No pagó matrícula,19528


## Función de preprocesado

Se eliminan columnas poco relevantes y se usa la codificación ordinal para columnas de tipo objeto

In [46]:
def preprocesar_datos(df, es_entrenamiento=True, mapeo_target=None, encoder=None):
  df = df.copy()

  columnas_a_eliminar = [
        'FAMI_TIENEINTERNET.1',
        'ESTU_PRIVADO_LIBERTAD',
        'FAMI_TIENELAVADORA',
        'FAMI_TIENEAUTOMOVIL',
        'PERIODO'
  ]

  for col in columnas_a_eliminar:
      if col in df.columns:
          del df[col]

  # Imputar valores nulos con moda
  for col in df.select_dtypes(include='object').columns:
      df[col].fillna(df[col].mode()[0], inplace=True)

  # Imputar valores nulos numéricos si es necesario
  for col in df.select_dtypes(include=['float64', 'int64']).columns:
      df[col].fillna(df[col].median(), inplace=True)

  # Codificación ordinal para educación
  educacion_orden = {
      'Ninguno': 0,
      'Primaria incompleta': 1,
      'Primaria completa': 2,
      'Secundaria (Bachillerato) incompleta': 3,
      'Secundaria (Bachillerato) completa': 4,
      'Técnica o tecnológica incompleta': 5,
      'Técnica o tecnológica completa': 6,
      'Educación profesional incompleta': 7,
      'Educación profesional completa': 8,
      'Postgrado': 9,
      'No sabe': -1,
      'No aplica': -2
  }

  for col in ['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE']:
    if col in df.columns:
        df[col] = df[col].map(educacion_orden)

  # Codificación ordinal para estrato
  estrato_map = {
      'Sin Estrato': 0,
      'Estrato 1': 1,
      'Estrato 2': 2,
      'Estrato 3': 3,
      'Estrato 4': 4,
      'Estrato 5': 5,
      'Estrato 6': 6
  }
  if 'FAMI_ESTRATOVIVIENDA' in df.columns:
    df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].map(estrato_map)

  # Codificación ordinal para horas de trabajo
  horas_map = {
      '0': 0,
      'Menos de 10 horas': 1,
      'Entre 11 y 20 horas': 2,
      'Entre 21 y 30 horas': 3,
      'Más de 30 horas': 4
  }
  if 'ESTU_HORASSEMANATRABAJA' in df.columns:
    df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].map(horas_map)

  # Codificación ordinal para matrícula
  matricula_map = {
      'No pagó matrícula': 0,
      'Menos de 500 mil': 1,
      'Entre 500 mil y menos de 1 millón': 2,
      'Entre 1 millón y menos de 2.5 millones': 3,
      'Entre 2.5 millones y menos de 4 millones': 4,
      'Entre 4 millones y menos de 5.5 millones': 5,
      'Entre 5.5 millones y menos de 7 millones': 6,
      'Más de 7 millones': 7
  }
  if 'ESTU_VALORMATRICULAUNIVERSIDAD' in df.columns:
    df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(matricula_map)

  # Codificación binaria para Si/No
  bin_cols = [col for col in df.columns if df[col].nunique() == 2 and df[col].dtype == 'object']
  for col in bin_cols:
    df[col] = df[col].map({'Si': 1, 'No': 0, 'S': 1, 'N': 0})

  # Codificación ordinal para el resto de categóricas usando OrdinalEncoder
  categ_cols = df.select_dtypes(include='object').columns.tolist()
  if 'RENDIMIENTO_GLOBAL' in categ_cols:
    categ_cols.remove('RENDIMIENTO_GLOBAL')
  if es_entrenamiento:
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    df[categ_cols] = encoder.fit_transform(df[categ_cols])
  else:
    df[categ_cols] = encoder.transform(df[categ_cols])

  # Codificar la variable objetivo si es entrenamiento
  if es_entrenamiento and 'RENDIMIENTO_GLOBAL' in df.columns:
    mapeo_target = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
    df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(mapeo_target)

  return df, mapeo_target, encoder

In [47]:
dtr_proc, mapeo_target, encoder = preprocesar_datos(dtr, es_entrenamiento=True)

/tmp/ipython-input-46-2469281481.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipython-input-46-2469281481.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

## Separación

In [48]:
X = dtr_proc.drop('RENDIMIENTO_GLOBAL', axis=1)
y = dtr_proc['RENDIMIENTO_GLOBAL']

## Carga y Preprocesado del archivo test

In [49]:
dts = pd.read_csv('test.csv')

In [50]:
dts.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


Se utiliza la misma función con los parámetros que guardamos cuando la aplicamos en el archivo train

In [51]:
dts_proc, _, _ = preprocesar_datos(dts, es_entrenamiento=False, mapeo_target=mapeo_target, encoder=encoder)

/tmp/ipython-input-46-2469281481.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipython-input-46-2469281481.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [52]:
X_test = dts_proc.drop(columns=['ID'])
X_test = X_test.reindex(columns=X.columns, fill_value=0)

In [53]:
X_test.head()

,ID,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4
0,0,943.0,5.0,1,1,3,1,6.0,1,1,2.0,0.328,0.219,0.317,0.247
1,0,12.0,1.0,4,3,2,1,4.0,0,1,6.0,0.227,0.283,0.296,0.324
2,0,456.0,4.0,3,0,3,1,3.0,0,1,4.0,0.285,0.228,0.294,0.247
3,0,229.0,27.0,3,3,1,0,1.0,0,0,1.0,0.160,0.408,0.217,0.294
4,0,14.0,3.0,4,2,2,1,4.0,0,1,4.0,0.209,0.283,0.306,0.286


# Solución final



## Optuna y validación cruzada

Se utiliza una función de optimización con optuna para identificar los parámetros más adecuados para el modelo escogido

In [54]:
# Función de optimización
def objective(trial):
  params = {
      'objective': 'multi:softmax',
      'num_class': 4,
      'n_estimators': trial.suggest_int('n_estimators', 100, 300),
      'max_depth': trial.suggest_int('max_depth', 3, 10),
      'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
      'subsample': trial.suggest_float('subsample', 0.5, 1.0),
      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
      'gamma': trial.suggest_float('gamma', 0, 5),
      'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
      'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
      'use_label_encoder': False,
      'eval_metric': 'mlogloss',
      'random_state': 42
  }

  pipeline = Pipeline([
      ("imputer", SimpleImputer(strategy="most_frequent")),
      ("clf", XGBClassifier(**params))
  ])

  # Validación cruzada estratificada
  cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
  scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy', n_jobs=-1)

  return scores.mean()

# Ejecutar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

print("Mejores parámetros encontrados:")
print(study.best_params)

[I 2025-07-04 02:12:43,898] A new study created in memory with name: no-name-3f4a6298-7c7b-4755-94dd-f76f7c602c44
[I 2025-07-04 02:14:06,385] Trial 0 finished with value: 0.4261602877657273 and parameters: {'n_estimators': 180, 'max_depth': 6, 'learning_rate': 0.12221599540493804, 'subsample': 0.5156695777809273, 'colsample_bytree': 0.7346266918580602, 'gamma': 2.132342976804625, 'reg_alpha': 0.3949495024304449, 'reg_lambda': 0.7607967182412069}. Best is trial 0 with value: 0.4261602877657273.
[I 2025-07-04 02:15:18,489] Trial 1 finished with value: 0.4265342947065209 and parameters: {'n_estimators': 218, 'max_depth': 6, 'learning_rate': 0.25609456525712576, 'subsample': 0.612712569017664, 'colsample_bytree': 0.883558658304195, 'gamma': 2.5324586149502943, 'reg_alpha': 0.6318961023036124, 'reg_lambda': 0.7130493026840999}. Best is trial 1 with value: 0.4265342947065209.
[I 2025-07-04 02:16:35,995] Trial 2 finished with value: 0.40612996197591755 and parameters: {'n_estimators': 174, 'm

Mejores parámetros encontrados:
{'n_estimators': 196, 'max_depth': 10, 'learning_rate': 0.1051462909804775, 'subsample': 0.7933562054246958, 'colsample_bytree': 0.585108364198306, 'gamma': 3.655861930473499, 'reg_alpha': 0.24364727695766483, 'reg_lambda': 0.09697504417918712}


## Entrenar modelo y realizar predicciones

In [55]:
# Crear pipeline con mejores parámetros encontrados por Optuna
best_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("clf", XGBClassifier(
        **study.best_params,
        objective='multi:softmax',
        num_class=4,
        use_label_encoder=False,
        eval_metric='mlogloss',
        random_state=42
    ))
])

# Entrenar pipeline con TODO el conjunto de entrenamiento
best_pipeline.fit(X, y)

# Predecir
y_preds = best_pipeline.predict(X_test)

# Revertir codificación
reverse_map = {v: k for k, v in mapeo_target.items()}
y_preds_labels = pd.Series(y_preds).map(reverse_map)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:50:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


## Generar archivo de solución para Kaggle

In [56]:
submission = pd.DataFrame({
    'ID': dts['ID'],
    'RENDIMIENTO_GLOBAL': y_preds_labels
})
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-bajo
2,499179,alto
3,782980,bajo
4,785185,bajo


In [57]:
submission.to_csv('submission9.csv', index=False)

In [58]:
!head submission9.csv

ID,RENDIMIENTO_GLOBAL
550236,bajo
98545,medio-bajo
499179,alto
782980,bajo
785185,bajo
58495,bajo
705444,alto
557548,alto
519909,bajo
